In [ ]:
%load_ext autoreload
%autoreload 2

#### Loading Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import statsmodels.api as sm
from statsmodels.tools import add_constant

In [ ]:
project_root = os.path.abspath('..')

# Adding the project root to my system path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data_loader import telco_data_loader
from src.data_processor import telcoDataCleaner

#### Loading and cleaning data

In [ ]:
# running the loader
raw_telco_data = telco_data_loader()

cleaner = telcoDataCleaner()

cleaned_data = cleaner.clean_data(raw_telco_data)

In [ ]:
len(cleaned_data['target'][cleaned_data["masks"]])

## Testing Models

Data prep

In [ ]:
# identifying predeictors columns with NA's

# Only selecting columns that aren't missing values
non_na_cols = []

for col in cleaned_data['predictors']:
    if cleaned_data['predictors'][col].isna().sum() == 0:
        non_na_cols.append(col)

# Adding constant to the predictors
reg_predictors = add_constant(cleaned_data['predictors'].copy(),
                              has_constant = 'raise',
                              prepend = True)

In [ ]:
log_reg_model = sm.Logit(endog = cleaned_data['target'][cleaned_data["masks"]],
                         exog = reg_predictors.loc[cleaned_data["masks"], :]).fit()

In [ ]:
print(log_reg_model.summary())